# The JuMP ecosystem for mathematical optimization: MOI

## JuliaCon 2018

## Juan Pablo Vielma
## MIT Sloan

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


In [2]:
using JuMP  
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver

## MOI and JuMP

In [3]:
model = Model()
@variable(model, x[1:2])
@constraint(model, linear1, 2*x[1] +   x[2] <= 1)  # save reference linear1 for this constraint
@constraint(model, 2*x[1] + 2*x[2] <= 1) 
@constraint(model, x[1]^2 + x[2]^2 <= 1)  
@constraint(model, [1;x] in SecondOrderCone())  # ||x|| <= 1
@objective(model, Max, x[1] + x[2]) 

x[1] + x[2]

Behind every JuMP model there is a MOI *backend* (used to be a MPB model)

In [4]:
typeof(backend(model))

MathOptInterface.Utilities.CachingOptimizer{MathOptInterface.AbstractOptimizer,MathOptInterface.Utilities.UniversalFallback{JuMP._MOIModel{Float64}}}

# MOI and JuMP variables

```julia
struct JuMP.VariableRef # MOI variable + owning model
    model::Model
    index::MOI.VariableIndex
end

struct MOI.VariableIndex  # Type-safe unique non-consecutive index
    value::Int64
end
```

In [5]:
xx = MOI.get(backend(model), MOI.ListOfVariableIndices()) 

2-element Array{MathOptInterface.VariableIndex,1}:
 MathOptInterface.VariableIndex(1)
 MathOptInterface.VariableIndex(2)

In [6]:
x[1].index == xx[1] && x[1].model == model

true

# MOI and JuMP constraints: First MOI constraints

In [7]:
model = Model()
@variable(model, x[1:2])
@constraint(model, linear1, 2*x[1] +   x[2] <= 1)  # save reference linear1 for this constraint
@constraint(model, 2*x[1] + 2*x[2] <= 1) 
@constraint(model, x[1]^2 + x[2]^2 <= 1)  
@constraint(model, [1;x] in SecondOrderCone()) 

L,Q,C = MOI.get(backend(model), MOI.ListOfConstraints())

3-element Array{Tuple{DataType,DataType},1}:
 (MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64})   
 (MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.LessThan{Float64})
 (MathOptInterface.VectorAffineFunction{Float64}, MathOptInterface.SecondOrderCone)     

# MOI constraints format 
$$
f(x) \in S
$$

were $f$ is an ``MOI.AbstractFunction`` and $S$ is an ``MOI.AbstractSet`` 
## Example:
JuMP:
```julia
@constraint(model, linear1, 2*x[1] +   x[2] <= 1)
```
``MOI.ScalarAffineFunction``-in-``MOI.LessThan``:

$$a^T x + b \in \{y:y\leq u\} \text{ for } a=(2,1)^T,\text{ }b=0 \text{ and }u=0$$ 

# More Constraint Examples

| Mathematical Constraint       | MOI Function                 | MOI Set        |
|-------------------------------|------------------------------|----------------|
| $a^Tx = b$                  | `ScalarAffineFunction`       | `EqualTo`      |
| $l \le a^Tx \le u$          | `ScalarAffineFunction`       | `Interval`     |
| $x_i \le u$                 | `SingleVariable`             | `LessThan`     |
| $Ax + b \in \mathbb{R}_+^n$ | `VectorAffineFunction`       | `Nonnegatives` |
| $\lVert Ax + b\rVert_2 \le c^Tx + d$                        | `VectorAffineFunction`       | `SecondOrderCone` |
| $y \exp (x/y) \le z, y > 0$  | `VectorOfVariables`       | `ExponentialCone`                  |
| $x \in \mathbb{R}^{d\left(d+1\right)/2}$,◹$(x)\in \text{PSD} \subseteq \mathbb{R}^{d\times d}$                                       | `VectorOfVariables`          | `PositiveSemidefiniteConeTriangle` |
| $x^TQx + a^Tx + b \ge 0$    | `ScalarQuadraticFunction`    | `GreaterThan`                 |
| $x_i \in \mathbb{Z}$                                                                     | `SingleVariable`    | `Integer`                          |
| $x_i \in \{0,1\}$                                                                        | `SingleVariable`    | `ZeroOne`                          |
| $x_i \in \{0\} \cup [l,u]$                                                               | `SingleVariable`    | `Semicontinuous`                   |
| At most one component of $x$ can be nonzero                                              | `VectorOfVariables` | `SOS1`                             |

MOI includes standard functions and sets, and allows for some extensions. See [MOI manual](http://www.juliaopt.org/MathOptInterface.jl/stable/index.html) for more details on [supported sets and functions](http://www.juliaopt.org/MathOptInterface.jl/stable/apimanual/#Standard-form-problem-1) and more [constraint examples](http://www.juliaopt.org/MathOptInterface.jl/stable/apimanual/#Constraints-by-function-set-pairs-1).

# MOI and JuMP constraints

```julia
struct JuMP.ConstraintRef{C} # MOI constraint + owning model
    model::Model
    index::C             # C = ConstraintIndex{F,S} for some F,S
end

struct MOI.ConstraintIndex{F,S} # Type-safe unique (within class) non-consecutive  index for class F-in-S
    value::Int64
end
```

```julia
@constraint(model, linear1, 2*x[1] +   x[2] <= 1)  # save reference linear1 for this constraint
@constraint(model, 2*x[1] + 2*x[2] <= 1) 
@constraint(model, x[1]^2 + x[2]^2 <= 1)  
@constraint(model, [1;x] in SecondOrderCone()) 
```

In [8]:
linear = MOI.get(backend(model), MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.LessThan{Float64}}())  #MOI.get(model.moibackend,MOI.ListOfConstraintIndices{L...}())

2-element Array{MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},1}:
 MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}(1)
 MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}(2)

# Example: Linear inequalities

JuMP:
```julia
@constraint(model, linear1, 2*x[1] +   x[2] <= 1)
```
``MOI.ScalarAffineFunction``-in-``MOI.LessThan``:

$$a^T x + b \in \{y:y\leq u\} \text{ for } a=(2,1)^T,\text{ }b=0 \text{ and }u=0$$ 

In [9]:
linear1.index == linear[1] && linear1.model == model

true

$a=(2,1)^T$:

In [10]:
MOI.get(backend(model), MOI.ConstraintFunction(),linear[1]).terms

2-element Array{MathOptInterface.ScalarAffineTerm{Float64},1}:
 MathOptInterface.ScalarAffineTerm{Float64}(2.0, MathOptInterface.VariableIndex(1))
 MathOptInterface.ScalarAffineTerm{Float64}(1.0, MathOptInterface.VariableIndex(2))

$b=0$:

In [11]:
MOI.get(backend(model), MOI.ConstraintFunction(),linear[1]).constant

0.0

$u=0$:

In [12]:
MOI.get(backend(model), MOI.ConstraintSet(),linear[1]).upper

1.0

## Similar for other constraints:

In [13]:
MOI.get(backend(model), MOI.ListOfConstraintIndices{MOI.ScalarQuadraticFunction{Float64}, MOI.LessThan{Float64}}())  #MOI.get(model.moibackend,MOI.ListOfConstraintIndices{Q...}())

1-element Array{MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64},MathOptInterface.LessThan{Float64}},1}:
 MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64},MathOptInterface.LessThan{Float64}}(3)

In [14]:
MOI.get(backend(model), MOI.ListOfConstraintIndices{MOI.VectorAffineFunction{Float64}, MOI.SecondOrderCone}())  #MOI.get(model.moibackend,MOI.ListOfConstraintIndices{C...}())

1-element Array{MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.SecondOrderCone},1}:
 MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.SecondOrderCone}(4)

## MOI Constraints

### Note: there are multiple ways to write the same constraint. For more details and discussions see [Constraint Bridges](#Constraint-Bridges).

## A Complete MOI Model

In [15]:
using MathOptInterface
const MOI = MathOptInterface
using GLPK

# Solves the binary-constrained knapsack problem:
# max c'x: w'x <= C, x binary using GLPK.

c = [1.0, 2.0, 3.0]
w = [0.3, 0.5, 1.0]
C = 3.2

num_variables = length(c)

optimizer = GLPK.Optimizer()

# Create the variables in the problem.
x = MOI.add_variables(optimizer, num_variables)

# Set the objective function.
objective_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(c, x), 0.0)
MOI.set(optimizer, MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(),
        objective_function)
MOI.set(optimizer, MOI.ObjectiveSense(), MOI.MAX_SENSE)

# Add the knapsack constraint.
knapsack_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(w, x), 0.0)
MOI.add_constraint(optimizer, knapsack_function, MOI.LessThan(C))

# Add integrality constraints.
for i in 1:num_variables
    MOI.add_constraint(optimizer, MOI.SingleVariable(x[i]), MOI.ZeroOne())
end

# All set!
MOI.optimize!(optimizer)

termination_status = MOI.get(optimizer, MOI.TerminationStatus())
obj_value = MOI.get(optimizer, MOI.ObjectiveValue())
if termination_status != MOI.OPTIMAL
    error("Solver terminated with status $termination_status")
end

@assert MOI.get(optimizer, MOI.ResultCount()) > 0

@assert MOI.get(optimizer, MOI.PrimalStatus()) == MOI.FEASIBLE_POINT

primal_variable_result = MOI.get(optimizer, MOI.VariablePrimal(), x)

@show obj_value
@show primal_variable_result

obj_value = 6.0
primal_variable_result = [1.0, 1.0, 1.0]


3-element Array{Float64,1}:
 1.0
 1.0
 1.0

## Model modifications

You can delete variables or constraints, modify constraints or transform constraints. Modifications are as efficient as allowed by the solver.

Example: Transform constraint

In [16]:
model = Model() 
@variable(model, x[1:2])
@constraint(model, socp, [1;x] in SecondOrderCone())  # ||x|| <= 1
@objective(model, Max, x[1] + x[2]) 

x[1] + x[2]

In [17]:
rsocp = MOI.transform(backend(model), socp.index, MOI.RotatedSecondOrderCone(3))

MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.RotatedSecondOrderCone}(2)

In [18]:
rsocp != socp

true

In [19]:
MOI.is_valid(backend(model), socp.index)

false

In [20]:
model = Model() 
@variable(model, x[1:2])
@constraint(model, linear1, 2 * x[1] +   x[2] <= 1)  # save reference linear1 for this constraint
@constraint(model, linear2, 2 * x[1] + 2 * x[2] <= 1) 
@constraint(model, quadratic, x[1]^2 + x[2]^2 <= 1)  
@constraint(model, socp, [1;x] in SecondOrderCone())  # ||x|| <= 1
@objective(model, Max, x[1] + x[2]) 

x[1] + x[2]

Delete constraint:

In [21]:
MOI.delete(backend(model), x[1].index)
MOI.delete(backend(model), linear1.index)

Modify constraints:

In [22]:
MOI.modify(backend(model), linear2.index, MOI.ScalarCoefficientChange(x[1].index, 3.0))

(MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}(2), MathOptInterface.ScalarAffineFunction{Float64}(MathOptInterface.ScalarAffineTerm{Float64}[ScalarAffineTerm{Float64}(2.0, VariableIndex(2)), ScalarAffineTerm{Float64}(3.0, VariableIndex(1))], 0.0), MathOptInterface.LessThan{Float64}(1.0))

### Also JuMP modifications

In [23]:
model = Model() 
@variable(model, x[1:2])
@constraint(model, linear1, 2 * x[1] +   x[2] <= 1)  # save reference linear1 for this constraint
@constraint(model, linear2, 2 * x[1] + 2 * x[2] <= 1) 
@constraint(model, quadratic, x[1]^2 + x[2]^2 <= 1)  
@constraint(model, socp, [1;x] in SecondOrderCone())  # ||x|| <= 1
@objective(model, Max, x[1] + x[2]) 

x[1] + x[2]

In [24]:
set_coefficient(linear1, x[1], 3)
linear1

linear1 : 3 x[1] + x[2] ≤ 1.0

In [25]:
delete(model,linear2)
delete(model,x[2])
model

A JuMP Model
Maximization problem with:
Variable: 1
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
`GenericQuadExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
`Array{GenericAffExpr{Float64,VariableRef},1}`-in-`MathOptInterface.SecondOrderCone`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: linear1, linear2, quadratic, socp, x

## MOI Attributes: For MOI models

In [26]:
model = GLPK.Optimizer()
var_index = MOI.add_variable(model)
constr_index = MOI.add_constraint(model,MOI.SingleVariable(var_index),MOI.LessThan(10.0))

MathOptInterface.ConstraintIndex{MathOptInterface.SingleVariable,MathOptInterface.LessThan{Float64}}(1)

for building models...

In [27]:
MOI.set(model, MOI.ObjectiveSense(),MOI.MAX_SENSE)
MOI.set(model, MOI.ObjectiveFunction{MOI.SingleVariable}(), MOI.SingleVariable(var_index))

and more ...

In [29]:
MOI.set(model, MOI.VariablePrimalStart(),var_index,5.0)
MOI.set(model, MOI.ConstraintDualStart(),constr_index,5.0)

MathOptInterface.UnsupportedAttribute{MathOptInterface.VariablePrimalStart}: MathOptInterface.UnsupportedAttribute{MathOptInterface.VariablePrimalStart}: Attribute MathOptInterface.VariablePrimalStart() is not supported by the the model.

## MOI Attributes: Also for JuMP models

In [30]:
model = Model()

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

some attributes have custom macro syntax

In [31]:
@variable(model, x >= 10.0, start = 5.0)
@objective(model, Max, x)
set_start_value(x, 3.0)

3.0

others use MOI functions

In [32]:
bound_ref = JuMP.LowerBoundRef(x).index
MOI.set(backend(model), MOI.ConstraintDualStart(),bound_ref,10.0)

10.0

## JuMP and MOI Solver Modes 

In [33]:
model = Model(with_optimizer(GLPK.Optimizer))
@variable(model, x >= 0)
@variable(model, y >= 0)
@constraint(model, inequality, x + y <= 1)
@objective(model, Max, x + y)

x + y

JuMP has 3 solver modes:

In [34]:
JuMP.ModelMode

Enum ModelMode:
AUTOMATIC = 0
MANUAL = 1
DIRECT = 2

Default mode is `Automatic`

In [35]:
JuMP.mode(model) == JuMP.AUTOMATIC

true

In [36]:
c = [1; 3; 5; 2] 

A= [
     1 1 9 5;
     3 5 0 8;
     2 0 6 13
    ]

b = [7; 3; 5] 

m, n = size(A); # m = number of rows of A, n = number of columns of A

# One Extreme: Automatic Mode = "Almost" Old JuMP

`CachingOptimizer` re-builds model when incremental modifications are not supported.
* Pros: No need to worry about what modifications are supported.
* Cons: No information about efficiency of modifications (may be re-creating the whole model at each `optimize` call)

**Note:** Automatic mode also provides automatic constraint transformations. See [Constraint Bridges](#Constraint-Bridges)

In [37]:
model = JuMP.Model(with_optimizer(GLPK.Optimizer, msg_lev = 0))
@variable(model, x[1:n] >= 0) 
@constraint(model, [i=1:m], sum(A[i,j]*x[j] for j in 1:n) == b[i])
@objective(model, Min, sum(c[j]*x[j] for j in 1:n)) 
@time begin
JuMP.optimize!(model) # solves the model
end
@show JuMP.termination_status(model) == MOI.OPTIMAL

  3.529461 seconds (13.61 M allocations: 691.330 MiB, 9.79% gc time)
JuMP.termination_status(model) == MOI.OPTIMAL = true


true

**Note:** 
```julia
with_optimizer(GLPK.Optimizer, msg_lev = 0)
```
builds a factory that can create instances of 
```julia
GLPK.Optimizer(msg_lev = 0)
```
to re-build the internal model (or copy it).

### The Other Extreme: Direct Mode

Direct connection to a solver. All data is stored with solver, MOI does not save or cache anything.
* Pros: Access to complete solver interfase (e.g. callbacks). Allowed modifications are as efficient as possible for the solver. 
* Cons: Cannot apply incremental modifications not supported by the solver. Cannot copy the model. [Constraint Bridges](#Constraint-Bridges) are not enabled by default.

In [38]:
model = JuMP.direct_model(GLPK.Optimizer(msg_lev = 0))
@variable(model, x[1:n] >= 0) # Models x >=0
@constraint(model, [i=1:m], sum(A[i,j]*x[j] for j in 1:n) == b[i])
@objective(model, Min, sum(c[j]*x[j] for j in 1:n)) 
@time begin
JuMP.optimize!(model) # solves the model
end
@show JuMP.termination_status(model) == MOI.OPTIMAL

  0.000053 seconds (7 allocations: 432 bytes)
JuMP.termination_status(model) == MOI.OPTIMAL = true


true

### Intermediate Mode: Manual Mode = Direct Control of `CachingOptimizer`

In [39]:
model = JuMP.Model(with_optimizer(GLPK.Optimizer, msg_lev = 4),caching_mode = MOIU.MANUAL)

@variable(model, x[1:n] >= 0) 
@constraint(model, [i=1:m], sum(A[i,j]*x[j] for j in 1:n) == b[i])
@objective(model, Min, sum(c[j]*x[j] for j in 1:n)) 

JuMP.optimize!(model)

AssertionError: AssertionError: m.state == ATTACHED_OPTIMIZER

Need to control `CachingOptimizer` directly.

 # Controlling `CachingOptimizer` 

In [40]:
copt = backend(model)
typeof(copt)

MathOptInterface.Utilities.CachingOptimizer{MathOptInterface.AbstractOptimizer,MathOptInterface.Utilities.UniversalFallback{JuMP._MOIModel{Float64}}}

In [41]:
MOIU.CachingOptimizerState

Enum MathOptInterface.Utilities.CachingOptimizerState:
NO_OPTIMIZER = 0
EMPTY_OPTIMIZER = 1
ATTACHED_OPTIMIZER = 2

In [42]:
MOIU.state(copt) == MOIU.EMPTY_OPTIMIZER

true

`EMPTY_OPTIMIZER` has not loaded the model. We can load it with `attachoptimizer!` and then we can solve:

In [43]:
MOIU.attach_optimizer(copt)

In [44]:
MOIU.state(copt) == MOIU.ATTACHED_OPTIMIZER

true

In [45]:
JuMP.optimize!(model)

GLPK Simplex Optimizer, v4.64
3 rows, 4 columns, 10 non-zeros
      0: obj =   0.000000000e+00 inf =   1.500e+01 (3)
      3: obj =   5.035196687e+00 inf =   0.000e+00 (0)
*     4: obj =   4.923076923e+00 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND


We can return to `EmptyOptimizer` with `resetoptimizer`:

In [46]:
MOIU.reset_optimizer(copt);
MOIU.state(copt) == MOIU.EMPTY_OPTIMIZER

true

# Constraint Bridges

## Constraint Bridges: Example 1

In [47]:
model = JuMP.Model()
@variable(model,x[1:10]);

Two-sided linear constraint as one `ScalarAffineFunction`-in-`Interval`:

In [48]:
@constraint(model, interval_inequality, -2 <= sum(x) <= 1)
#interval_inequality.index

interval_inequality : x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] ∈ [-2.0, 1.0]

Two-sided linear constraint as two `ScalarAffineFunction`-in-`LessThan`:

In [49]:
@constraint(model, lower_bound, -sum(x) <= 2)
#lower_bound.index
@constraint(model, upper_bound,  sum(x) <= 1)
#upper_bound.index

upper_bound : x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] ≤ 1.0

## Constraint Bridges: Example 2

In [50]:
model = JuMP.Model()
@variable(model,x)
@variable(model,y)
@variable(model,t);

Quadratic inequality as `ScalarQuadraticFunction`-in-`LessThan`:

In [51]:
@constraint(model,quadratic, x^2 + y^2 <= t)
#quadratic.index

quadratic : x² + y² - t ≤ 0.0

Quadratic inequality as `VectorAffineFunction`-in-`SecondOrderCone`:

In [52]:
@constraint(model, conic, [(1+t)/2; (1-t)/2; x; y] in SecondOrderCone())
#conic.index

conic : [0.5 t + 0.5, -0.5 t + 0.5, x, y] ∈ MathOptInterface.SecondOrderCone(4)

# [Constraint Bridges](http://www.juliaopt.org/MathOptInterface.jl/stable/apimanual.html#Constraint-bridges-1) provide automatic transformation rules.

For instance, [Automatic Mode](#One-Extreme:-Automatic-Mode-=-"Almost"-Old-JuMP) actually adds two intermadiate layers: 
1. `CachingOptimizer` for incremental modifications, and
2. `LazyBridgeOptimizer <: AbstractBridgeOptimizer` for default automatic transformations

`LazyBridgeOptimizer` can be disabled with the `bridge_constraints = false` keyword argument.

# Example for `bridge_constraints`

ECOS does not support quadratic constraints so it fails without a bridge

In [53]:
using ECOS
model = JuMP.Model(with_optimizer(ECOS.Optimizer), bridge_constraints = false)
@variable(model,x)
@variable(model,y)
@variable(model,t)
@constraint(model,quadratic, x^2 + y^2 <= t)

ErrorException: Constraints of type MathOptInterface.ScalarQuadraticFunction{Float64}-in-MathOptInterface.LessThan{Float64} are not supported by the solver, try using `bridge_constraints=true` in the `JuMP.Model` constructor if you believe the constraint can be reformulated to constraints supported by the solver.

In [54]:
model = JuMP.Model(with_optimizer(ECOS.Optimizer), bridge_constraints = true)
@variable(model,x)
@variable(model,y)
@variable(model,t)
@constraint(model,quadratic, x^2 + y^2 <= t)

quadratic : x² + y² - t ≤ 0.0

## JuMP and MOI

In [55]:
model = Model(with_optimizer(GLPK.Optimizer))
@variable(model, x >= 0)
@variable(model, y >= 0)
@constraint(model, inequality, x + y <= 1)
@objective(model, Max, x + y)

x + y

In [56]:
@macroexpand @variable(model, x >= 0)

quote
    #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/macros.jl:278 =#
    (JuMP._valid_model)(model, :model)
    #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/macros.jl:279 =#
    begin
        #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/macros.jl:336 =#
        (JuMP._error_if_cannot_register)(model, :x)
        #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/macros.jl:340 =#
        #458###516 = begin
                #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/macros.jl:294 =#
                let
                    #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/macros.jl:300 =#
                    #458###516 = (JuMP.add_variable)(model, (JuMP.build_variable)(getfield(JuMP, Symbol("#_error#66")){Tuple{Symbol,Expr}}((:model, :(x >= 0))), (JuMP.VariableInfo)(true, 0, false, NaN, false, NaN, false, NaN, false, false)), "x")
                    #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/macros.jl:301 =#
                    #458###516
                end
      

`@variable(model, x >= 0)`:

1. Create JuMP/MOI variable and set its attributes:
```julia
x = JuMP.add)variable(model,[...],"x")
```
2. Save JuMP variable in symbol-keyed dictionary:
```julia
((JuMP.object_dictionary)(model))[:x] = x 
```

In [57]:
@macroexpand @constraint(model, inequality, x + y <= 1)

quote
    #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/macros.jl:278 =#
    (JuMP._valid_model)(model, :model)
    #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/macros.jl:279 =#
    begin
        #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/macros.jl:336 =#
        (JuMP._error_if_cannot_register)(model, :inequality)
        #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/macros.jl:340 =#
        #464###517 = begin
                #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/macros.jl:294 =#
                let
                    #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/macros.jl:300 =#
                    begin
                        #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/macros.jl:620 =#
                        begin
                            #460#q = (JuMP.Val){false}()
                            #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/macros.jl:410 =#
                            begin
                                begin
            

`@constraint(model, inequality, x + y <= 1)`:

1. Create JuMP/MOI constraint (function and set):
```julia
inequality = JuMP.add_constraint(model,[...],"inequality")
``` 
2. Save JuMP constraint in symbol-keyed dictionary:
```julia
((JuMP.object_dictionary)(model))[:inequality] = inequality
``` 

#### JuMP stores data only on MOI

```julia
function moi_add_constraint(model::MOI.ModelLike, f::MOI.AbstractFunction,
                            s::MOI.AbstractSet)
    [...]
    return MOI.add_constraint(model, f, s)
end


function add_constraint(model::Model, c::AbstractConstraint, name::String="")
    [...]
    cindex = moi_add_constraint(backend(model), moi_function(c), moi_set(c))
    cshape = shape(c)
    if !(cshape isa ScalarShape) && !(cshape isa VectorShape)
        model.shapes[cindex] = cshape
    end
    cref = ConstraintRef(model, cindex, cshape)
    if !isempty(name)
        set_name(cref, name)
    end
    return cref
end
```
JuMP constraint:
```julia
struct ConstraintRef{M <: AbstractModel, C, Shape <: AbstractShape}
    model::M
    index::C
    shape::Shape
end
```

We can recover constraints by their "name" stored in MOI:

```julia
function constraint_by_name(model::Model, name::String)
    index = MOI.get(backend(model), MOI.ConstraintIndex, name)
    if index isa Nothing
        return nothing
    else
        return constraint_ref_with_index(model, index)
    end
end

function name(cr::ConstraintRef{Model,<:_MOICON})
    return MOI.get(cr.model, MOI.ConstraintName(), cr)::String
end
```

In [58]:
constraint_by_name(model, "inequality") == inequality

true

In [59]:
"inequality" == name(inequality)

true

also for variables:

In [60]:
variable_by_name(model, "x") == x

true

In [61]:
"x" == name(x)

true

#### JuMP does store dictionary to recover variables/constraints by symbol

In [62]:
model[:x] == x

true

In [63]:
model[:inequality] == inequality

true

usefull when working with multiple models with same names:

In [64]:
model1 = Model(with_optimizer(GLPK.Optimizer))
@variable(model1, x >= 0)
@variable(model1, y >= 0)
@constraint(model1, inequality, x + y <= 1)
@objective(model1, Max, x + y)

model2 = Model(with_optimizer(GLPK.Optimizer))
@variable(model2, x >= 0)
@variable(model2, y >= 0)
@constraint(model2, inequality, 2x + y <= 1)
@objective(model2, Max, x + y)

x + y

`inequality` for `model1` was overwritten by `inequality` for `model2`: 

In [65]:
inequality

inequality : 2 x + y ≤ 1.0

we can recover `inequality` for `model1` from `model1[...]`:

In [66]:
model1[:inequality]

inequality : x + y ≤ 1.0

### A recap of the four types of variables/constraints in JuMP

Julia variables v/s mathematical optimization variables?! 

In [67]:
model = Model(with_optimizer(GLPK.Optimizer))
@variable(model, x[1:3] >= 0, base_name="Xvar")
@variable(model, y >= 0)
@constraint(model, inequality, sum(x) + y <= 1)
@objective(model, Max, sum(x) + y)

Xvar[1] + Xvar[2] + Xvar[3] + y

MOI variable: a unique index within a model

In [68]:
y.index

MathOptInterface.VariableIndex(4)

In [69]:
x[2].index

MathOptInterface.VariableIndex(2)

MOI variable name: a unique string associated to a MOI variable.

In [70]:
name(y)

"y"

In [71]:
name(x[2])

"Xvar[2]"

In [72]:
set_name(x[3],"lastXvar")

In [73]:
x

3-element Array{VariableRef,1}:
 Xvar[1] 
 Xvar[2] 
 lastXvar

Single JuMP variable: Reference to a MOI variable in a model. A Julia variable of type `JuMP.VariableRef`

In [74]:
typeof(y)

VariableRef

In [75]:
y.index

MathOptInterface.VariableIndex(4)

In [76]:
y.model == model

true

Container of JuMP variables: well... a container of `JuMP.VariableRef` 

In [77]:
typeof(x)

Array{VariableRef,1}

### Interesting Examples:

In [78]:
model = Model(with_optimizer(GLPK.Optimizer))
x = @variable(model, [1:3])

3-element Array{VariableRef,1}:
 noname
 noname
 noname

In [79]:
x

3-element Array{VariableRef,1}:
 noname
 noname
 noname

In [80]:
model = Model(with_optimizer(GLPK.Optimizer))
@variable(model, [1:3], base_name=string(rand()))

3-element Array{VariableRef,1}:
 0.7529760151049645[1]
 0.8728570610412967[2]
 0.5619090361506274[3]

In [81]:
using DataStructures
myvariables = Stack{JuMP.VariableRef}()
model = Model(with_optimizer(GLPK.Optimizer))
push!(myvariables,@variable(model, y >= 0))
push!.([myvariables],@variable(model, z[1:3] >= 0))
pop!(myvariables)

z[3]

In [82]:
@show myvariables

myvariables = Stack{VariableRef}(Deque [VariableRef[y, z[1], z[2]]])


Stack{VariableRef}(Deque [VariableRef[y, z[1], z[2]]])